In [1]:
import numpy as np
from scipy.linalg import *
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame,Series
import math
import time

In [20]:
Re = 400
rho = 1
L = 0.1
V = 1
mu = rho*V*L/Re
w = 1

In [24]:
meshx = 5
deltax = L/meshx

meshy = 5
deltay = L/meshy

mesht = 2 #para o primeiro tempo as velocidades e a pressão são zero
deltat = 1e10

In [27]:
P = np.zeros(((mesht,meshx,meshy)))
u = np.zeros(((mesht,meshx,meshy)))
v = np.zeros(((mesht,meshx,meshy)))

In [23]:
# Massa do Volume de Controle
Mp=rho*deltax*deltay*w
# Termos Difusivos
De_u=mu*deltay*w/deltax
De_v=mu*deltay*w/deltax
Dw_u=mu*deltay*w/deltax
Dw_v=mu*deltay*w/deltax
Dn_u=mu*deltax*w/deltay
Dn_v=mu*deltax*w/deltay
Ds_u=mu*deltax*w/deltay
Ds_v=mu*deltax*w/deltay

#a_p_o_u=(M_p/deltat)*ones(NTx,NTy)
#a_p_o_v=(M_p/deltat)*ones(NTx,NTy)



In [33]:
#CDS
x_plot = np.zeros(meshx)
y_plot = np.zeros(meshy)
t_plot = np.zeros(mesht)

x_plot = np.linspace(deltax/2,L-deltax/2,meshx)
y_plot = np.linspace(deltay/2,L-deltay/2,meshy)

Me_u = np.zeros((meshx,meshy))
Mw_u = np.zeros((meshx,meshy))
Mn_u = np.zeros((meshx,meshy))
Ms_u = np.zeros((meshx,meshy))

Me_v = np.zeros((meshx,meshy))
Mw_v = np.zeros((meshx,meshy))
Mn_v = np.zeros((meshx,meshy))
Ms_v = np.zeros((meshx,meshy))


eps = 1e-6
erro = 1
for t in range(1,mesht):
    
    while (erro > eps): #arrumar dps
        for x in range(1,meshx-1):
            for y in range(1,meshy-1):
                Me_u[x,y] = rho*(u[t,x,y]+u[t,x+1,y])/2*deltay*w
                Mw_u[x,y] = rho*(u[t,x,y]+u[t,x-1,y])/2*deltay*w
                Mn_u[x,y] = rho*(v[t,x,y]+v[t,x+1,y])/2*deltax*w
                Ms_u[x,y] = rho*(v[t,x,y-1]+v[t,x+1,y-1])/2*deltax*w
                
                Me_v[x,y] = rho*(u[t,x,y]+u[t,x,y+1])/2*deltay*w
                Mw_v[x,y] = rho*(u[t,x-1,y+1]+u[t,x-1,y])/2*deltay*w
                Mn_v[x,y] = rho*(v[t,x,y]+v[t,x,y+1])/2*deltax*w
                Ms_v[x,y] = rho*(v[t,x,y]+v[t,x,y-1])/2*deltax*w

                ae_u[x,y] = De_u[x,y] - Me_u[x,y]
                aw_u[x,y] = Dw_u[x,y] + Mw_u[x,y]
                an_u[x,y] = Dn_u[x,y] - Mn_u[x,y]
                as_u[x,y] = Ds_u[x,y] + Ms_u[x,y]
                ap0_u[x,y] = Mp/deltat
                b_u[x,y] =ap0_u[x,y]*u[t-1,x,y]
                ap_u[x,y] = ae_u[x,y] + aw_u[x,y] + an_u[x,y] + as_u[x,y] + ap0_u[x,y]
                
                ae_v[x,y] = De_v[x,y] - Me_v[x,y]
                aw_v[x,y] = Dw_v[x,y] + Mw_v[x,y]
                an_v[x,y] = Dn_v[x,y] - Mn_v[x,y]
                as_v[x,y] = Ds_v[x,y] + Ms_v[x,y]
                ap0_v[x,y] = Mp/deltat
                b_v[x,y] =ap0_v[x,y]*v[t-1,x,y]
                ap_v[x,y] = ae_v[x,y] + aw_v[x,y] + an_v[x,y] + as_v[x,y] + ap0_v[x,y]
                
        for y in range(meshy):
            ap[meshx-1,y] = 1
                